In [28]:
using StochasticPrograms
using LinearAlgebra
using Statistics
using HiGHS
using SparseArrays
using GLPK
using JuMP

In [30]:
# Constructions des 2 vecteurs de tailles 3^5=243x1 pour chaque scénario possible
demands1 = [150 160 170]
demands2 = [100 120 135]
demands3 = [250 270 300]
demands4 = [300 325 350]
demands5 = [600 700 800]
probs1 = [0.25 0.5 0.25]
probs2 = [0.25 0.5 0.25]
probs3 = [0.25 0.5 0.25]
probs4 = [0.3 0.4 0.3]
probs5 = [0.3 0.4 0.3]
scenarios = collect.(Iterators.product(demands1,demands2,demands3,demands4,demands5) |> collect)
prob = prod.(Iterators.product(probs1, probs2, probs3, probs4, probs5) |> collect)

# Les constants 
factories = 3
markets = 5
trans_cost = [2.49 5.21 3.76 4.85 2.07; 1.46 2.54 1.83 1.86 4.76; 3.26 3.08 2.60 3.76 4.45]
prod_cost = 14
total_cost = trans_cost .+ prod_cost
price = 24
waste_cost = 4
cap = [500 450 650]

# J'ai choisi GLPK ici car c'est plus vite sur mon PC et le problème peut être résolu linéairement
#solver = HiGHS.Optimizer
solver = GLPK.Optimizer

GLPK.Optimizer

In [32]:
# Construction du problème de 1ere étape
function firststage()
    m = Model(solver)
    @variable(m, Ship[i in 1:factories, j in 1:markets] >= 0)
    @variable(m, θ[i in 1:243])
    @constraint(m, [i in 1:factories], sum(Ship[i,j] for j in 1:markets) <= cap[i])
    
    # Pas encore theta car il n'y pas encore de contrainte sur la variable
    @objective(m, Min,
        sum(total_cost[i,j]*Ship[i,j] for i in 1:factories, j in 1:markets)
    )
    
    return m, Ship, θ
end

firststage (generic function with 1 method)

In [34]:
# Problème master, c'est juste first stage + theta
# Dans le cas multi cut, le nombre de thetas = nombre de scénarios
function master_objective(m::Model, Ship, θ)
    @objective(m, Min,
        sum(total_cost[i,j]*Ship[i,j] for i in 1:factories, j in 1:markets)
        + sum(θ[i] for i in 1:243)
    )
    return m
end

master_objective (generic function with 1 method)

In [36]:
function secondstageCore(Ship, demands)
    m = Model()

    @variable(m, Sales[j in 1:markets] >= 0)
    @variable(m, Waste[j in 1:markets] >= 0)

    # Contrainte Waste_j + Sales_j = sum(Ship[i,j]) for i -> 3 usines pour chaque j -> 3 matrice identité négative (-Tx)
    # Contrainte Sales < Demand -> pas de x -> matrice zéro 
    unit_mat = sparse([1,2,3,4,5],[1,2,3,4,5],[-1,-1,-1,-1,-1])
    T = [unit_mat unit_mat unit_mat; spzeros(5,15)]
    
    # Contrainte Waste_j + Sales_j = sum(Ship[i,j]) for i pas de constant -> vecteur 0
    # Contrainte Sales < Demand -> constant est la demande -> vecteur demande    
    h = sparse([zeros(5); demands])


    recourseConstraints = []

    return m, Sales, Waste, recourseConstraints, h, T
end 

secondstageCore (generic function with 1 method)

In [38]:
function secondstage(Ship, demands)
    m, Sales, Waste, recourseConstraints, h, T = secondstageCore(Ship, demands)


    # Contrainte de recours
    for j = 1:markets    
        push!(recourseConstraints, @constraint(m, Waste[j] + Sales[j] == sum(Ship[i,j] for i in 1:factories)))
    end

    for j = 1:markets
        push!(recourseConstraints, @constraint(m, Sales[j] <= demands[j]))
    end
    
    # Q(x*,E)
    @objective(m, Min, -sum(price * Sales[j] for j in 1:markets)
                           + sum(waste_cost * Waste[j] for j in 1:markets))

    set_optimizer(m, solver)
    optimize!(m)
    
    return m, recourseConstraints, h, T
end

secondstage (generic function with 1 method)

In [40]:
# Fonction pour générer la coupe feasibility
# Notre problème n'a pas besoin de cette coupe alors c'est moins important
function secondstagefeasibility(Ship, demands)
    m, Sales, Waste, recourseConstraints, h, T = secondstageCore(Ship, demands)

    nb = 10
    @variable(m, w[1:nb] >= 0)
    for j = 1:markets    
        push!(recourseConstraints, @constraint(m, Waste[j] + Sales[j] + w[j] == sum(Ship[i,j] for i in 1:factories)))
    end

    for j = 1:markets
        push!(recourseConstraints, @constraint(m, Sales[j] - w[(j+markets)] <= demands[j]))
    end

    @objective(m, Min, sum(w[i] for i in 1:nb))

    set_optimizer(m, solver)
    optimize!(m)
    σ = dual.(recourseConstraints)  # since the objective function is a minimization, we have the correct sign.

    return σ, h, T
end

secondstagefeasibility (generic function with 1 method)

In [48]:
# Vérifier si tout theta_k > Q_k -> Convergence -> Arrêter l'algo
function stop(expected_Q, θ, k, tol = 1e-10)
    nmax = 200  # circuit breaker
    counting = 0
    for i in 1:243 
        if ((θ[i] > expected_Q[i])|| isapprox(θ[i]-expected_Q[i], 0.0; atol=tol, rtol=0))
            counting+=1
        end
    end
    return (counting >= 243 || k==nmax)
end

stop (generic function with 2 methods)

In [50]:
function lshaped_multicut(scenarios, prob, verbose=false)
    nScenarios = 243
    
    k = 0       # iteration index
    nfcuts = 0  # number of feasibility cuts
    nocuts = 0  # number of optimality cuts
    
    first, Ship, θ = firststage()
    n = factories*markets

    Q_list = [+Inf for i in 1:nScenarios]
    valθ = [-Inf for i in 1:nScenarios]

    # Commence de l'algo L-shaped
    while (!stop(Q_list, valθ, k))
        Q_list = []
        k += 1

        if verbose
            println(first)
        end

        #Step 1
        optimize!(first)
        status = termination_status(first)
        
        if (status != MOI.OPTIMAL)
            print("sus")
            println("Error: status ", status)
            return status, Ship, first
        end
        
        # status == MOI.OPTIMAL
        xsol = value.(Ship)
        if k!=1
            valθ = value.(θ)
        end
        E = zeros(n)'
        e = 0.0
        Q = 0.0
        
        # Solve the second-stage programs
        # Step 2 + 3
        for iter = 1:nScenarios
            p = prob[iter]
            m, scstrs, h, T = secondstage(xsol, scenarios[iter])
            status = termination_status(m)

            if (status == MOI.INFEASIBLE)
                # Step 2
                # Code never enter this section
                if dual_status(m) == MOI.INFEASIBILITY_CERTIFICATE
                    # If the solver emits a infeasibility certificate, we can rely on it
                    # Note: this has been tested with HiGHS only
                    # Get dual ray from constraints (infeasibility certificate):
                    # For each constraint, try to extract its dual multiplier

                    # We cannot use all_constraints as JuMP reorganize the constraints by blocks of the same type
                    # σ = dual.(all_constraints(m; include_variable_in_set_constraints=false))
                    σ = dual.(scstrs)

                    # The sign of the dual variables depend of the sense of the optimization (min or max)
                    # Julia uses the conic duality (see https://jump.dev/MathOptInterface.jl/stable/background/duality/)
                    # As a consequence, we have to take the opposite of the dual variables in case of a minimization
                    sense = objective_sense(m)
                    if (sense == MOI.MAX_SENSE)
                        σ *= -1
                    end
                else
                    # No infeasibility certificate available
                    # We explictly solve the feasibility second-stage problem
                    σ, h, T = secondstagefeasibility(xsol, collect(scenarios[i]))
                end

                # Build a feasibility cut
                E = σ'*T
                @constraint(first, sum(E[(5*(i-1)+j)]*Ship[i,j] for i in 1:factories, j in 1:markets) >= σ'*h)
                nfcuts += 1
                break;
            elseif (status == MOI.OPTIMAL)
                # Step 3
                # Build the optimality cut component
                π = dual.(scstrs)
                E = p*(π'*T)
                e = p*(dot(π,h))
                
                Q = p*objective_value(m)
                push!(Q_list,Q)

                # Step 4 Ajout des coupes
                if (nocuts == 0)
                    # add θ in the problem
                    @constraint(first, con, sum(E[(5*(i-1)+j)]*Ship[i,j] for i in 1:factories, j in 1:markets) + θ[iter] >= e)
                    master_objective(first, Ship, θ)
                else
                    check = valθ[iter]
                    if (check < Q)
                        @constraint(first, sum(E[(5*(i-1)+j)]*Ship[i,j] for i in 1:factories, j in 1:markets) + θ[iter] >= e)
                    end
                end
                nocuts += 1        
            else
                println("Error second-stage resolution: status ", status)
                return status, Ship, first
            end
        end    
    end
    optimize!(first)
    return k,nfcuts,nocuts,Ship, first
end

lshaped_multicut (generic function with 2 methods)

In [52]:
k,nfcuts,nocuts,Ship, firstst = lshaped_multicut(scenarios, prob)

(8, 0, 1944, VariableRef[Ship[1,1] Ship[1,2] … Ship[1,4] Ship[1,5]; Ship[2,1] Ship[2,2] … Ship[2,4] Ship[2,5]; Ship[3,1] Ship[3,2] … Ship[3,4] Ship[3,5]], A JuMP Model
Minimization problem with:
Variables: 258
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1457 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 15 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: Ship, con, θ)

In [54]:
objective_value(firstst)

-10792.999999999989

In [56]:
value.(Ship)

3×5 Matrix{Float64}:
   0.0    0.0    0.0    0.0  500.0
 150.0    0.0    0.0  300.0    0.0
   0.0  100.0  270.0    0.0  100.0